Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic

In [20]:
batch_size = 128
l2_param = 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
           tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
         l2_param * tf.nn.l2_loss(weights)
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [21]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 24.254116
Minibatch accuracy: 15.6%
Validation accuracy: 24.5%
Minibatch loss at step 500: 2.805560
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 1000: 1.216121
Minibatch accuracy: 79.7%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 0.734462
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.549371
Minibatch accuracy: 85.9%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 0.863143
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.691984
Minibatch accuracy: 82.0%
Validation accuracy: 82.0%
Test accuracy: 89.0%


### Neural Network

In [28]:
num_hidden_nodes = 1024
l2_param = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_i = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_i = tf.Variable(tf.zeros([num_hidden_nodes]))

  weights_h = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_h = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_i) + biases_i)
  visible_logits = tf.matmul(hidden_logits, weights_h) + biases_h
  loss = tf.reduce_mean(
           tf.nn.softmax_cross_entropy_with_logits(visible_logits, tf_train_labels)) + \
         l2_param * (tf.nn.l2_loss(weights_i) + tf.nn.l2_loss(weights_h))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(visible_logits)
  valid_prediction = tf.nn.softmax(
                       tf.matmul(
                         tf.nn.relu(
                           tf.matmul(
                             tf_valid_dataset, 
                             weights_i) + 
                         biases_i), 
                       weights_h) + 
                     biases_h)
  test_prediction =  tf.nn.softmax(
                       tf.matmul(
                         tf.nn.relu(
                           tf.matmul(
                             tf_test_dataset, 
                             weights_i) + 
                         biases_i), 
                       weights_h) + 
                     biases_h)

In [29]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 631.468018
Minibatch accuracy: 6.2%
Validation accuracy: 33.3%
Minibatch loss at step 500: 199.625931
Minibatch accuracy: 76.6%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 117.778854
Minibatch accuracy: 77.3%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 69.101280
Minibatch accuracy: 84.4%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 41.351887
Minibatch accuracy: 87.5%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 25.313046
Minibatch accuracy: 88.3%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 15.592288
Minibatch accuracy: 88.3%
Validation accuracy: 86.9%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [47]:
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_i = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_i = tf.Variable(tf.zeros([num_hidden_nodes]))

  weights_h = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_h = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_i) + biases_i)
  visible_logits = tf.matmul(hidden_logits, weights_h) + biases_h
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(visible_logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(visible_logits)
  valid_prediction = tf.nn.softmax(
                       tf.matmul(
                         tf.nn.relu(
                           tf.matmul(
                             tf_valid_dataset, 
                             weights_i) + 
                         biases_i), 
                       weights_h) + 
                     biases_h)
  test_prediction =  tf.nn.softmax(
                       tf.matmul(
                         tf.nn.relu(
                           tf.matmul(
                             tf_test_dataset, 
                             weights_i) + 
                         biases_i), 
                       weights_h) + 
                     biases_h)

In [48]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % ((train_labels.shape[0] / 1000) - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 293.809662
Minibatch accuracy: 7.8%
Validation accuracy: 28.7%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.0%
Test accuracy: 73.0%


/home/diogo/bin/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/diogo/bin/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [70]:
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_i = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_i = tf.Variable(tf.zeros([num_hidden_nodes]))

  weights_h = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_h = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_i) + biases_i)

  # Dropout.
  hidden_logits_dropout = tf.nn.dropout(hidden_logits, np.ones_like(hidden_logits) * 0.5)

  visible_logits = tf.matmul(hidden_logits_dropout, weights_h) + biases_h
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(visible_logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(visible_logits)
  valid_prediction = tf.nn.softmax(
                       tf.matmul(
                         tf.nn.relu(
                           tf.matmul(
                             tf_valid_dataset, 
                             weights_i) + 
                         biases_i), 
                       weights_h) + 
                     biases_h)
  test_prediction =  tf.nn.softmax(
                       tf.matmul(
                         tf.nn.relu(
                           tf.matmul(
                             tf_test_dataset, 
                             weights_i) + 
                         biases_i), 
                       weights_h) + 
                     biases_h)

In [71]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % ((train_labels.shape[0] / 1000) - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 474.595306
Minibatch accuracy: 7.0%
Validation accuracy: 31.7%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.5%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.1%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Test accuracy: 78.8%


/home/diogo/bin/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/diogo/bin/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [102]:
num_hidden1_nodes = 128
num_hidden2_nodes = 64
num_hidden3_nodes = 32
num_steps = 3001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_i = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden1_nodes]))
  biases_i = tf.Variable(tf.zeros([num_hidden1_nodes]))

  weights_h1 = tf.Variable(
    tf.truncated_normal([num_hidden1_nodes, num_hidden2_nodes]))
  biases_h1 = tf.Variable(tf.zeros([num_hidden2_nodes]))
    
  weights_h2 = tf.Variable(
    tf.truncated_normal([num_hidden2_nodes, num_hidden3_nodes]))
  biases_h2 = tf.Variable(tf.zeros([num_hidden3_nodes]))
    
  weights_h3 = tf.Variable(
    tf.truncated_normal([num_hidden3_nodes, num_labels]))
  biases_h3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1_logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_i) + biases_i)
  hidden1_logits_dropout = tf.nn.dropout(hidden1_logits, np.ones_like(hidden1_logits) * 0.5)
    
  hidden2_logits = tf.nn.relu(tf.matmul(hidden1_logits_dropout, weights_h1) + biases_h1)
  hidden2_logits_dropout = tf.nn.dropout(hidden2_logits, np.ones_like(hidden2_logits) * 0.5)
    
  hidden3_logits = tf.nn.relu(tf.matmul(hidden2_logits_dropout, weights_h2) + biases_h2)
  hidden3_logits_dropout = tf.nn.dropout(hidden3_logits, np.ones_like(hidden3_logits) * 0.5)

  visible_logits = tf.matmul(hidden3_logits_dropout, weights_h3) + biases_h3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(visible_logits, tf_train_labels))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.03, global_step, num_steps, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)#, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(visible_logits)
    
  valid_prediction = tf.nn.softmax(
                       tf.matmul(
                         tf.nn.relu(
                           tf.matmul(
                             tf.nn.relu(
                               tf.matmul(
                                 tf.nn.relu(
                                   tf.matmul(
                                     tf_valid_dataset, 
                                     weights_i) + 
                                   biases_i), 
                                 weights_h1) + 
                               biases_h1), 
                             weights_h2) + 
                           biases_h2), 
                         weights_h3) + 
                       biases_h3)

  test_prediction = tf.nn.softmax(
                       tf.matmul(
                         tf.nn.relu(
                           tf.matmul(
                             tf.nn.relu(
                               tf.matmul(
                                 tf.nn.relu(
                                   tf.matmul(
                                     tf_test_dataset, 
                                     weights_i) + 
                                   biases_i), 
                                 weights_h1) + 
                               biases_h1), 
                             weights_h2) + 
                           biases_h2), 
                         weights_h3) + 
                       biases_h3)

In [103]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4980.844727
Minibatch accuracy: 8.6%
Validation accuracy: 11.0%


InvalidArgumentError: ReluGrad input is not finite. : Tensor had Inf values
	 [[Node: gradients/Relu_2_grad/Relu_2/CheckNumerics = CheckNumerics[T=DT_FLOAT, message="ReluGrad input is not finite.", _device="/job:localhost/replica:0/task:0/cpu:0"](add_2)]]
Caused by op 'gradients/Relu_2_grad/Relu_2/CheckNumerics', defined at:
  File "/home/diogo/bin/anaconda3/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/diogo/bin/anaconda3/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/diogo/bin/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/diogo/bin/anaconda3/lib/python3.4/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/home/diogo/bin/anaconda3/lib/python3.4/site-packages/ipykernel/kernelapp.py", line 403, in start
    ioloop.IOLoop.instance().start()
  File "/home/diogo/bin/an